In [22]:
import findspark
findspark.init()
findspark.find()

'C:\\apps\\opt\\spark-3.3.1-bin-hadoop3'

,transaction_type,transaction_id
825,GENERIC_VAT_ON_PSITEK_COMMISSION,202
826,GENERIC_SD_COMMISSION,203
827,GENERIC_VAT_ON_SD_COMMISSION,204
828,GENERIC_DEVICE_COMMISSION,205
829,GENERIC_VAT_ON_DEVICE_COMMISSION,206


In [38]:
"""
Created on Wednesday Feb 1 2023
@author: Olabode Ajayi
"""
#import required modules and pandas library
from pyspark.sql import SparkSession
import pyodbc
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

def normalize_space(my_s):
    """Return s stripped of leading/trailing whitespace
    and with internal runs of whitespace replaced by a single SPACE"""
    # remove all the occurrences of the TRANSACTION_TYPE_ character:
    # This should be a str method :-(
    my_new_string = my_s.replace("TRANSACTION_TYPE_", "")
    return ' '.join(my_new_string.split())


with open('kazang_transaction_types.py','r') as file:
    content = file.readlines()

# Create Dictonary and read line by line content of the file
my_dict = {}

for line in content:
    split = line.split('=')
    result= [normalize_space(i) for i in split]
    #print(result)
    my_dict[result[0]] = result[1]

    
# creating a Dataframe object from a list 
# of tuples of key, value pair
df = pd.DataFrame(list(my_dict.items()))
# Rename the column by assigning .columns attribute:
df.columns = ['transaction_type', 'transaction_primary_type_id']
df.to_csv('transaction_type.csv',sep=';',index=False) # Use semi-colon to seperate data and export to CSV format


# initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("Python ETL script for TEST") \
    .master("local[*]")\
    .config("spark.driver.memory", '8g')\
    .config("spark.sql.ansi.enabled ",True)\
    .config("spark.driver.extraClassPath", "C:\apps\opt\spark-3.3.1-bin-hadoop3\jars\sqljdbc42.jar") \
    .getOrCreate()

#Create spark configuration object
conf = SparkConf()
conf.setMaster("local").setAppName("My app")
 
#Create spark context and sparksession
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
 
#read csv file in a dataframe
df = spark.read.csv(path="transaction_type.csv", header = True, sep = ";")

 
#set variable to be used to connect the database

table = "[ArchiveRawData].[dbo].[transaction_type]"
database = "ArchiveRawData"
user = "kaz"
password  = "Password12345"
 
#write the dataframe into a sql table
df.write.mode("overwrite") \
    .format("jdbc") \
    .option("url", f"jdbc:sqlserver://localhost:1433;databaseName={database};encrypt=true;trustServerCertificate=true;") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()


query_string= """
            SELECT *
            FROM [ArchiveRawData].[dbo].[transaction_type]
            """

#Read ETL Meta Data
etl_meta_data_staging = spark.read\
    .format("jdbc") \
    .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
    .option("url", "jdbc:sqlserver://127.0.0.1:1433;databaseName={"+database+"};encrypt=true;trustServerCertificate=true;") \
    .option("query", query_string) \
    .option("user", user) \
    .option("password", password) \
    .load().show()

+--------------------+---------------------------+
|    transaction_type|transaction_primary_type_id|
+--------------------+---------------------------+
|             DEPOSIT|                          0|
|          WITHDRAWAL|                          1|
|        RESET_PROFIT|                          2|
|       LOCAL_CONTENT|                          3|
|    RELATED_TRANSFER|                          4|
|      STOCK_PURCHASE|                          5|
|          STOCK_SALE|                          6|
|   DEVICE_COMMISSION|                          7|
| CUSTOMER_COMMISSION|                          8|
|            REVERSAL|                          9|
|        VAT_ON_SALES|                         10|
|             JOURNAL|                         11|
|             PAYMENT|                         12|
|                 CDF|                         13|
|VAT_ON_CUSTOMER_C...|                         14|
|          VAT_ON_CDF|                         15|
|        VAT_ON_STOCK|         